In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
from shapely.geometry import Polygon
import pickle

Key meta data

In [2]:
root_path = "D:/GeoData/"
Main_CRS = "EPSG:27700"

Shapefiles

In [3]:
Shape_Loc = {
    'All_GB' : root_path + "Countries__December_2019__Boundaries_UK_BFC-shp/Countries__December_2019__Boundaries_UK_BFC.shp",
    'National_Parks' : root_path + "National_Parks__December_2019__GB_BFE-shp/National_Parks__December_2019__GB_BFE.shp",
    'LocalAuthorities' : root_path + "Local_Authority_Districts__May_2020__Boundaries_UK_BFC-shp/Local_Authority_Districts__May_2020__Boundaries_UK_BFC.shp",
    'LSOA' : root_path + "Lower_Layer_Super_Output_Area__December_2011__EW_BSC_V2-shp/Lower_Layer_Super_Output_Area__December_2011__EW_BSC_V2.shp",
    'GreenSpace' : root_path + "opgrsp_essh_gb/OS Open Greenspace (ESRI Shape File) GB/data/GB_GreenspaceSite.shp",
    'Rivers' : root_path + "oprvrs_essh_gb/data/WatercourseLink.shp",
    'Railway_Lines' : root_path + "strtgi_essh_gb/data/railway_line.shp",
    'Woodland_Region' : root_path + "strtgi_essh_gb/data/woodland_region.shp",
    'Urban_Region' : root_path + "strtgi_essh_gb/data/urban_region.shp",
    'Foreshor_Region' : root_path + "strtgi_essh_gb/data/foreshor_region.shp",
    'Ferry_Line' : root_path + "strtgi_essh_gb/data/ferry_line.shp",
    'Coastline' : root_path + "strtgi_essh_gb/data/coastline.shp" ,
    'Gridlines' : root_path + "strtgi_essh_gb/data/gridlines.shp",
    'Lakes' : root_path + "strtgi_essh_gb/data/lakes_region.shp"}

In [4]:
All_Maps = dict((k, gpd.read_file(v).to_crs(Main_CRS)) for k, v in Shape_Loc.items())

In [5]:
file = os.listdir("D:\GeoData\oproad_essh_gb\data")

path = [os.path.join("D:\GeoData\oproad_essh_gb\data", i) for i in file if "RoadLink.shp" in i]

All_Maps['Road_Link'] = gpd.GeoDataFrame(pd.concat([gpd.read_file(i) for i in path], 
                        ignore_index=True), crs=Main_CRS)

In [6]:
path = [os.path.join("D:\GeoData\oproad_essh_gb\data", i) for i in file if "MotorwayJunction.shp" in i]

All_Maps['MotorwayJunctions'] = gpd.GeoDataFrame(pd.concat([gpd.read_file(i) for i in path], 
                        ignore_index=True), crs=Main_CRS)

In [7]:
All_Maps['Motorway'] = All_Maps['Road_Link'][All_Maps['Road_Link']['class'] == "Motorway"]
All_Maps['A_Roads'] = All_Maps['Road_Link'][All_Maps['Road_Link']['class'] == "A Road "]
All_Maps['B_Roads'] = All_Maps['Road_Link'][All_Maps['Road_Link']['class'] == "B Road"]
All_Maps['Other_roads'] = All_Maps['Road_Link'][~All_Maps['Road_Link']["class"].isin([ "Motorway", "A Road", "B Road"])]

#del All_Maps['Road_Link']

NSPL Data

In [8]:
NSPL_Data = pd.read_pickle(root_path + 'WorkingData/' + 'NSPL_Data.pickle')
All_Maps['NSPL_gdf'] = gpd.GeoDataFrame(NSPL_Data, geometry=gpd.points_from_xy(NSPL_Data.oseast1m, NSPL_Data.osnrth1m), crs=Main_CRS)

del NSPL_Data

Land Registry Data

In [9]:
LandReg = pd.read_pickle(root_path + 'WorkingData/' + 'LandReg.pickle')
All_Maps['LandReg_gdf'] = gpd.GeoDataFrame(LandReg, geometry=gpd.points_from_xy(LandReg.oseast1m, LandReg.osnrth1m), crs=Main_CRS)

del LandReg

There is a lot of mapping data here. My computer isn't that powerful so we need to subset the data. I've chosen a list of points and the code below clips the data to squares around these points

Functions for clipping and saving the maps

In [11]:
def MakePolygon(centre, sq_sz, inp_crs):
    mnx = centre[0]-(sq_sz/2)
    mny = centre[1]-(sq_sz/2)
    mxx = centre[0]+(sq_sz/2)
    mxy = centre[1]+(sq_sz/2)
    polygon =Polygon([(mnx, mny), (mnx,mxy), (mxx,mxy), (mxx,mny), (mnx, mny)])
    poly_gdf = gpd.GeoDataFrame([1], geometry=[polygon], crs=inp_crs)
    return poly_gdf

In [12]:
def save_obj(name , obj, extra=""):
    with open(root_path + 'WorkingData/' + extra + name + '.pkl', 'wb+') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [13]:
def load_obj(name ):
    with open(root_path + 'WorkingData/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [14]:
def ClipAndSave(name, centre, sq_sz, crs, extra):
    print(name)
    poly = MakePolygon(centre, sq_sz, crs)
    maps = dict((k, gpd.clip(v, poly)) for k, v in All_Maps.items())
    save_obj(name , maps, extra)
    return maps

Clip and save the maps for specific locations. Save at 5km^2, 10km^2, 20km^2 regions.

In [15]:
save_obj('All_Maps' , All_Maps)

In [18]:
KeyLocations = {'Hawkshead': [335329 , 498064],
                'South_London' : [535493, 171250],
                'Loch_Lomond' : [238450 , 690986],
                'Furness' : [326877 , 474372],
                'Croydon' : [532621 , 165313],
                'StPauls' : [532052 , 181145],
                'ElephantCastle' : [531958 , 179077],
                'M4_junct' : [519377 , 178360],
                'M6_Junctions' : [357659 , 425462],
                'Kendal': [353086 , 492882],
                'Milford Haven': [190649 , 205747],
                'Birmingham':[409295 , 290319],
                'Euston':[529947 , 182697]}

In [19]:
ClippedMaps_5k = dict((k, ClipAndSave(k, v, 5000, Main_CRS, "Clipped5k_")) for k, v in KeyLocations.items())
save_obj('ClippedMaps_5k' , ClippedMaps_5k)

Sydenham


Exception ignored on calling ctypes callback function: <function Index._create_idx_from_stream.<locals>.py_next_item at 0x0000021EFB1D2550>
Traceback (most recent call last):
  File "C:\Users\richa\Anaconda3\envs\LandReg_py\lib\site-packages\rtree\index.py", line 1047, in py_next_item
    p_id[0], coordinates, obj = next(stream_iter)
  File "C:\Users\richa\Anaconda3\envs\LandReg_py\lib\site-packages\geopandas\sindex.py", line 326, in <genexpr>
    (i, item.bounds, None)
  File "C:\Users\richa\Anaconda3\envs\LandReg_py\lib\site-packages\shapely\geometry\point.py", line 123, in bounds
    xy = self.coords[0]
  File "C:\Users\richa\Anaconda3\envs\LandReg_py\lib\site-packages\shapely\coords.py", line 70, in __getitem__
    self._update()
  File "C:\Users\richa\Anaconda3\envs\LandReg_py\lib\site-packages\shapely\coords.py", line 46, in _update
    self._cseq = lgeos.GEOSGeom_getCoordSeq(self.__p__._geom)
KeyboardInterrupt: 


Furness


KeyboardInterrupt: 

In [ ]:
ClippedMaps_10k = dict((k, ClipAndSave(k, v, 10000, Main_CRS, "Clipped10k_")) for k, v in KeyLocations.items())
save_obj('ClippedMaps_10k' , ClippedMaps_10k)